# Notebook d'exploration des données issues des pipelines SmartTech :
- delta_bronze : fichier Delta issue du pipeline en direct depuis les fichiers json
- delta_bronze_kafka : fichier Delta issue du pipeline en sortie de Kafka
- delta_silver_kafka : fichier Delta issue de la transformation de delta_bronze_kafka


In [1]:
##########################################################
###### SCRIPT POUR CONFIGURER SPARK AVEC DELTA LAKE ######
##########################################################

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_timestamp
from pathlib import Path
from delta import configure_spark_with_delta_pip


builder = (
    SparkSession.builder.appName("SmartTech_Exploration")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config(
        "spark.sql.catalog.spark_catalog",
        "org.apache.spark.sql.delta.catalog.DeltaCatalog",
    )
)


spark = configure_spark_with_delta_pip(builder).getOrCreate()

print(f"Spark version : {spark.version}")
print("Connecteur Delta chargé avec succès.")


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/12/17 14:54:45 WARN Utils: Your hostname, MacBook-Pro.local, resolves to a loopback address: 127.0.0.1; using 10.0.21.53 instead (on interface en0)
25/12/17 14:54:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/Users/jean-thomasmiquelot/kDrive/PROGRAMMATION/simplon/Simplon_projets/esther_spark_streaming/.venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /Users/jean-thomasmiquelot/.ivy2.5.2/cache
The jars for the packages stored in: /Users/jean-thomasmiquelot/.ivy2.5.2/jars
io.delta#delta-spark_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-86c7ffab-2f18-449e-b1ac-bd26f7942c76;1.0
	confs: [default]
	found io.delta#delta-spark_2.13;4.0.0 in central
	found io.delta#delta-storage;4.0.0 in central
	found org.antlr#antl

Spark version : 4.0.1
Connecteur Delta chargé avec succès.


# Exploration Table Delta Pipeline Classique

In [2]:
# Charger les données Delta Lake depuis le répertoire bronze
PROJECT_ROOT = Path.cwd().parent
BRONZE_PATH = PROJECT_ROOT / "data" / "out" / "delta_bronze"
df_bronze = spark.read.format("delta").load(str(BRONZE_PATH))

In [9]:
# Afficher les 20 premières lignes du DataFrame bronze
df_bronze.show(20, truncate=False)

+-------------------+-----------------+--------+-----+------------------+------+----+---------------+
|timestamp          |device_id        |building|floor|type              |value |unit|_corrupt_record|
+-------------------+-----------------+--------+-----+------------------+------+----+---------------+
|2025-01-12 09:27:25|sensor-temp-003  |B       |3    |temperature       |21.6  |°C  |NULL           |
|2025-01-12 09:27:28|sensor-temp-003  |A       |3    |temperature       |18.6  |°C  |NULL           |
|2025-01-12 09:27:29|sensor-temp-003  |B       |2    |temperature       |24.1  |°C  |NULL           |
|2025-01-12 09:27:30|sensor-energy-010|B       |3    |energy_consumption|172.7 |kWh |NULL           |
|2025-01-12 09:27:31|sensor-temp-001  |A       |2    |temperature       |18.2  |°C  |NULL           |
|2025-01-12 09:28:18|sensor-hum-002   |B       |3    |humidity          |59.1  |%   |NULL           |
|2025-01-12 09:28:20|sensor-temp-001  |A       |1    |temperature       |26.5  |°C

In [ ]:
# Créer une vue temporaire pour exécuter des requêtes SQL
df_bronze.createOrReplaceTempView("bronze_table")

In [6]:
# Exemple de requête SQL pour explorer les données
spark.sql("SELECT * from bronze_table LIMIT 10").show()

+-------------------+-----------------+--------+-----+------------------+------+----+---------------+
|          timestamp|        device_id|building|floor|              type| value|unit|_corrupt_record|
+-------------------+-----------------+--------+-----+------------------+------+----+---------------+
|2025-01-12 09:27:25|  sensor-temp-003|       B|    3|       temperature|  21.6|  °C|           NULL|
|2025-01-12 09:27:28|  sensor-temp-003|       A|    3|       temperature|  18.6|  °C|           NULL|
|2025-01-12 09:27:29|  sensor-temp-003|       B|    2|       temperature|  24.1|  °C|           NULL|
|2025-01-12 09:27:30|sensor-energy-010|       B|    3|energy_consumption| 172.7| kWh|           NULL|
|2025-01-12 09:27:31|  sensor-temp-001|       A|    2|       temperature|  18.2|  °C|           NULL|
|2025-01-12 09:38:21|  sensor-temp-001|       B|    1|       temperature|  20.3|  °C|           NULL|
|2025-01-12 09:38:23|  sensor-temp-001|       B|    2|       temperature|  23.3|  

In [8]:
# Compter le nombre total d'enregistrements dans la table bronze
# Possibilité de facilement modifier cette explorations SQL
sql_query = "SELECT COUNT(*) AS total_records FROM bronze_table"
spark.sql(sql_query).show()

+-------------+
|total_records|
+-------------+
|          500|
+-------------+



# Exploration Table Delta Pipeline avec Kafka

In [4]:
PROJECT_ROOT = Path.cwd().parent
BRONZE_KAFKA_PATH = PROJECT_ROOT / "data" / "out" / "delta_bronze_kafka"
SILVER_KAFKA_PATH = PROJECT_ROOT / "data" / "out" / "delta_silver_kafka"

In [5]:
df_bronze_kafka = spark.read.format("delta").load(str(BRONZE_KAFKA_PATH))
df_silver_kafka = spark.read.format("delta").load(str(SILVER_KAFKA_PATH))

## Exploration delta_bronze_kafka

In [6]:
df_bronze_kafka.show(20, truncate=False)

25/12/17 14:56:31 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------------------+-----------------+--------+-----+------------------+-----+----+---------------+
|timestamp          |device_id        |building|floor|type              |value|unit|_corrupt_record|
+-------------------+-----------------+--------+-----+------------------+-----+----+---------------+
|2025-01-12 09:39:53|sensor-hum-002   |B       |2    |humidity          |44.2 |%   |NULL           |
|2025-01-12 09:40:07|sensor-temp-001  |A       |1    |temperature       |26.9 |°C  |NULL           |
|2025-01-12 09:40:23|sensor-hum-002   |B       |2    |humidity          |37.9 |%   |NULL           |
|2025-01-12 09:40:28|sensor-temp-003  |B       |1    |temperature       |19.3 |°C  |NULL           |
|2025-01-12 09:40:47|sensor-hum-002   |B       |3    |humidity          |40.5 |%   |NULL           |
|2025-01-12 09:41:05|sensor-co2-020   |A       |1    |co2               |744.0|ppm |NULL           |
|2025-01-12 09:41:22|sensor-temp-001  |A       |1    |temperature       |24.3 |°C  |NULL   

In [ ]:
# afficher en pandas (plus propre)
# possible seulement si le dataset est petit
df_bronze_kafka.limit(500).toPandas()

,timestamp,device_id,building,floor,type,value,unit,_corrupt_record
0,2025-01-12 09:18:14,sensor-co2-020,A,2,co2,1008.000000,ppm,None
1,2025-01-12 09:18:23,sensor-hum-002,A,2,humidity,36.200001,%,None
2,2025-01-12 09:18:37,sensor-temp-003,A,3,temperature,27.799999,°C,None
3,2025-01-12 09:18:53,sensor-co2-020,B,3,co2,694.000000,ppm,None
4,2025-01-12 09:19:08,sensor-temp-003,A,2,temperature,21.500000,°C,None
...,...,...,...,...,...,...,...,...
395,2025-01-12 09:31:56,sensor-hum-002,B,1,humidity,30.600000,%,None
396,2025-01-12 09:37:53,sensor-hum-002,A,1,humidity,30.900000,%,None
397,2025-01-12 09:38:04,sensor-temp-003,B,3,temperature,21.299999,°C,None
398,2025-01-12 09:23:06,sensor-co2-020,A,3,co2,788.000000,ppm,None


In [ ]:
df_bronze_kafka.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- device_id: string (nullable = true)
 |-- building: string (nullable = true)
 |-- floor: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- value: float (nullable = true)
 |-- unit: string (nullable = true)
 |-- _corrupt_record: string (nullable = true)



In [ ]:
df_bronze_kafka.describe().show()

+-------+---------------+--------+-----------------+-----------+------------------+----+---------------+
|summary|      device_id|building|            floor|       type|             value|unit|_corrupt_record|
+-------+---------------+--------+-----------------+-----------+------------------+----+---------------+
|  count|            400|     400|              400|        400|               400| 400|              0|
|   mean|           NULL|    NULL|             1.88|       NULL|242.06100019454956|NULL|           NULL|
| stddev|           NULL|    NULL|0.829046618556127|       NULL| 375.6933593663134|NULL|           NULL|
|    min| sensor-co2-020|       A|                1|        co2|              18.2|   %|           NULL|
|    max|sensor-temp-003|       B|                3|temperature|            1199.0|  °C|           NULL|
+-------+---------------+--------+-----------------+-----------+------------------+----+---------------+



In [14]:
df_bronze_kafka.count()

200

In [ ]:
df_bronze_kafka.createOrReplaceTempView("kafka_bronze_table")

In [ ]:
spark.sql("SELECT * from kafka_bronze_table LIMIT 10").show()

+-------------------+---------------+--------+-----+-----------+------+----+---------------+
|          timestamp|      device_id|building|floor|       type| value|unit|_corrupt_record|
+-------------------+---------------+--------+-----+-----------+------+----+---------------+
|2025-01-12 09:34:35| sensor-co2-020|       B|    3|        co2| 817.0| ppm|           NULL|
|2025-01-12 09:34:36|sensor-temp-003|       B|    3|temperature|  23.2|  °C|           NULL|
|2025-01-12 09:34:40|sensor-temp-003|       B|    1|temperature|  19.9|  °C|           NULL|
|2025-01-12 09:34:42| sensor-co2-020|       B|    2|        co2|1051.0| ppm|           NULL|
|2025-01-12 09:34:44| sensor-hum-002|       B|    2|   humidity|  39.5|   %|           NULL|
|2025-01-12 09:42:44| sensor-co2-020|       B|    3|        co2| 571.0| ppm|           NULL|
|2025-01-12 09:42:48| sensor-hum-002|       A|    3|   humidity|  42.7|   %|           NULL|
|2025-01-12 09:42:50| sensor-co2-020|       A|    3|        co2| 922.0

## Exploration delta_silver_kafka

In [15]:
df_silver_kafka.show(20, truncate=False)

+-------------------+-----------------+--------+-----+------------------+------+----+
|timestamp          |device_id        |building|floor|type              |value |unit|
+-------------------+-----------------+--------+-----+------------------+------+----+
|2025-01-12 09:29:54|sensor-energy-010|b       |1    |energy_consumption|177.3 |kwh |
|2025-01-12 09:30:22|sensor-temp-001  |b       |2    |temperature       |25.7  |°c  |
|2025-01-12 09:26:40|sensor-temp-001  |a       |1    |temperature       |18.6  |°c  |
|2025-01-12 09:37:07|sensor-energy-010|b       |2    |energy_consumption|171.1 |kwh |
|2025-01-12 09:42:00|sensor-energy-010|b       |3    |energy_consumption|181.4 |kwh |
|2025-01-12 09:32:50|sensor-energy-010|a       |2    |energy_consumption|169.9 |kwh |
|2025-01-12 09:18:37|sensor-temp-003  |a       |3    |temperature       |27.8  |°c  |
|2025-01-12 09:19:08|sensor-temp-003  |a       |2    |temperature       |21.5  |°c  |
|2025-01-12 09:38:04|sensor-temp-003  |b       |3    |

In [16]:
# afficher en pandas (plus propre)
# possible seulement si le dataset est petit
df_silver_kafka.toPandas()

,timestamp,device_id,building,floor,type,value,unit
0,2025-01-12 09:28:44,sensor-temp-001,a,2,temperature,25.000000,°c
1,2025-01-12 09:29:40,sensor-temp-001,a,2,temperature,21.299999,°c
2,2025-01-12 09:29:54,sensor-energy-010,b,1,energy_consumption,177.300003,kwh
3,2025-01-12 09:30:22,sensor-temp-001,b,2,temperature,25.700001,°c
4,2025-01-12 09:26:40,sensor-temp-001,a,1,temperature,18.600000,°c
...,...,...,...,...,...,...,...
95,2025-01-12 09:31:23,sensor-hum-002,a,3,humidity,31.299999,%
96,2025-01-12 09:38:17,sensor-hum-002,b,2,humidity,44.000000,%
97,2025-01-12 09:34:32,sensor-temp-001,a,3,temperature,27.700001,°c
98,2025-01-12 09:22:01,sensor-hum-002,a,2,humidity,32.200001,%


In [19]:
df_silver_kafka.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- device_id: string (nullable = true)
 |-- building: string (nullable = true)
 |-- floor: string (nullable = true)
 |-- type: string (nullable = true)
 |-- value: float (nullable = true)
 |-- unit: string (nullable = true)



In [ ]:
df_silver_kafka.describe().show()

In [8]:
df_silver_kafka.createOrReplaceTempView("kafka_silver_table")
spark.sql("SELECT * from kafka_silver_table LIMIT 10").show()

+-------------------+-----------------+--------+-----+------------------+------+----+
|          timestamp|        device_id|building|floor|              type| value|unit|
+-------------------+-----------------+--------+-----+------------------+------+----+
|2025-01-12 09:29:54|sensor-energy-010|       b|    1|energy_consumption| 177.3| kwh|
|2025-01-12 09:30:22|  sensor-temp-001|       b|    2|       temperature|  25.7|  °c|
|2025-01-12 09:26:40|  sensor-temp-001|       a|    1|       temperature|  18.6|  °c|
|2025-01-12 09:37:07|sensor-energy-010|       b|    2|energy_consumption| 171.1| kwh|
|2025-01-12 09:42:00|sensor-energy-010|       b|    3|energy_consumption| 181.4| kwh|
|2025-01-12 09:18:37|  sensor-temp-003|       a|    3|       temperature|  27.8|  °c|
|2025-01-12 09:43:00|   sensor-hum-002|       a|    3|          humidity|  33.5|   %|
|2025-01-12 09:38:35|   sensor-co2-020|       a|    1|               co2|1163.0| ppm|
|2025-01-12 09:30:43|  sensor-temp-003|       b|    2|

In [12]:
# Compter le nombre total d'enregistrements dans la table silver kafka
# Possibilité de facilement modifier cette explorations SQL
silver_sql_query = "SELECT COUNT(*) AS total_records FROM kafka_silver_table"
spark.sql(silver_sql_query).show()

+-------------+
|total_records|
+-------------+
|          100|
+-------------+



In [10]:
spark.sql("SELECT DISTINCT (unit) as unit FROM kafka_silver_table").show()

+----+
|unit|
+----+
| kwh|
|  °c|
| ppm|
|   %|
+----+

